# Cleaning NYC high school data and exploring the fairness of the SAT  

The SAT, or Scholastic Aptitude Test, is a test that high school seniors in the U.S. take every year.  Colleges use the SAT to determine which students to admit.  High average SAT scores are usually indicative of a good school.  

New York City has published data on [student SAT scores](https://data.cityofnewyork.us/Education/2012-SAT-Results/f9bf-2cp4) by high school, along with additional related data sets:  

* SAT scores by school - SAT scores for each high school in New York City  
* High school directory - Various descriptive information for each high school in New York City (i.e. geographic coordinates)  
* Class size - Information on class size for each school  
* AP test results - Advanced Placement (AP) exam results for each high school (passing an optional AP exam in a particular subject can earn a student college credit in that subject)  
* Graduation outcomes - The percentage of students who graduated, and other outcome information  
* Demographics - Demographic information for each school  
* School survey - Surveys of parents, teachers, and students at each school  

New York City has a significant immigrant population and is very diverse, so comparing demographic factors such as race, income, and gender with SAT scores is a good way to determine whether the SAT is a fair test.  For example, if certain racial groups consistently perform better on the SAT, we would have some evidence that the SAT is unfair.  

I am going to clean and consolidate the datasets into one clean dataset so it can be used for further exploratory data analysis, visualization and predictive modeling.  

(cut pand paste this to "storytelling" and "visualization" project") ... and explore if the SAT is unfair to certain demographic groups or students in certain locations of NYC.  

This is the approach I am going to take...  

Here is a summary of my results...

# Reading in the csv data  

We will read each comma separated value (csv) data file into a Pandas Dataframe and then store them in a dictionary named "data".  This will provide quick and efficient access to the different csv datasets by storing them all in one data structure.  

We will read in and explore the text (txt) survey data files later.

In [1]:
import pandas as pd
pd.options.display.max_columns = 100  # So dataframe columns will not be condensed

# Create a list of the csv data files
data_files = [
    "ap_2010.csv",
    "class_size.csv",
    "demographics.csv",
    "graduation.csv",
    "hs_directory.csv",
    "sat_results.csv"
]

# Read each csv file into a dataframe and store in a dictionary
data = {}
for file in data_files:
    dataframe = pd.read_csv("../data/raw/{0}".format(file))  # apply the relative file locations
    key = file.replace(".csv", "")
    data[key] = dataframe
    
# View the first 5 rows of the "ap_2010" dataframe
data["ap_2010"].head()

,DBN,SchoolName,AP Test Takers,Total Exams Taken,Number of Exams with scores 3 4 or 5
0,01M448,UNIVERSITY NEIGHBORHOOD H.S.,39,49,10
1,01M450,EAST SIDE COMMUNITY HS,19,21,s
2,01M515,LOWER EASTSIDE PREP,24,26,24
3,01M539,"NEW EXPLORATIONS SCI,TECH,MATH",255,377,191
4,02M296,High School of Hospitality Management,s,s,s


# Exploring the "SAT" data  

We are interested in exploring relationships between the SAT data and the other datasets. We will look at the first 5 rows of the "sat_results" dataframe to see what we can learn on initial inspection.  

Referencing the "SAT_Results_Data_Dictionary" in the "references" folder of this project, each row provides the average SAT scores for each high school in NYC.  Each high school has a unique identification code called a "DBN" (district, borough, school number).  The scores are broken down into the 3 main sections of the test: Critical Reading, Math and Writing.  

We will explore any interesting correlations between this data and the other dataframes.

In [2]:
# View the first 5 rows of the "sat_results" dataframe
data["sat_results"].head()

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355,404,363
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383,423,366
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377,402,370
3,01M458,FORSYTH SATELLITE ACADEMY,7,414,401,359
4,01M509,MARTA VALLE HIGH SCHOOL,44,390,433,384


# Exploring the remaining data  

Similarly, we will look at the first 5 rows of each of the remaining dataframes to see what we can learn on initial inspection.  

### Exploring the "advanced placement" data  

Referencing the "AP_2010_Data_Dictionary" in the "references" folder of this project, this dataset provides advanced placement (AP) test results data for each NYC high school.  We can use this data to find any correlation between SAT results and AP results.

Each row represents various AP test results statistics for each NYC high school.  We can use the DBN column to join this data with the other dataframes.

In [3]:
# View the first 5 rows of the "ap_2010" dataframe
data["ap_2010"].head()

,DBN,SchoolName,AP Test Takers,Total Exams Taken,Number of Exams with scores 3 4 or 5
0,01M448,UNIVERSITY NEIGHBORHOOD H.S.,39,49,10
1,01M450,EAST SIDE COMMUNITY HS,19,21,s
2,01M515,LOWER EASTSIDE PREP,24,26,24
3,01M539,"NEW EXPLORATIONS SCI,TECH,MATH",255,377,191
4,02M296,High School of Hospitality Management,s,s,s


### Exploring the "class size" data  

Referencing the "Class_Size_Data_Dictionary" in the "references" folder of this project, this dataset provides average class sizes for each school, by grade and program type for grades 5-9 and 9-12, aggregated by program type and core course.  We can use this data to find any correlation between SAT results and class size.

Each row represents a specific grade level and program type for that school.  We will have to filter for the rows that contain the grade levels and program types that are most suitable for our analysis.

In [4]:
# View the first 5 rows of the "class_size" dataframe
data["class_size"].head()

,CSD,BOROUGH,SCHOOL CODE,SCHOOL NAME,GRADE,PROGRAM TYPE,CORE SUBJECT (MS CORE and 9-12 ONLY),CORE COURSE (MS CORE and 9-12 ONLY),SERVICE CATEGORY(K-9* ONLY),NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,DATA SOURCE,SCHOOLWIDE PUPIL-TEACHER RATIO
0,1,M,M015,P.S. 015 Roberto Clemente,0K,GEN ED,-,-,-,19.0,1.0,19.0,19.0,19.0,ATS,NaN
1,1,M,M015,P.S. 015 Roberto Clemente,0K,CTT,-,-,-,21.0,1.0,21.0,21.0,21.0,ATS,NaN
2,1,M,M015,P.S. 015 Roberto Clemente,01,GEN ED,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN
3,1,M,M015,P.S. 015 Roberto Clemente,01,CTT,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN
4,1,M,M015,P.S. 015 Roberto Clemente,02,GEN ED,-,-,-,15.0,1.0,15.0,15.0,15.0,ATS,NaN


There is also no DBN (district, borough, school number) column in this dataframe.  However, according to the "Class_Size_Data_Dictionary", the "CSD" column represents the district and the "school code" column represents the borough and school number.  This means we can create a DBN column by combining the "CSD" and "school code" columns.  If "CSD" is only a single digit, we need to add a leading "0" to make the DBN consistent with the other dataframes.  This will allow us to join this data with the other dataframes.

To confirm, we will take the DBN from the first row of the "sat_results" dataframe (above). That DBN is "01M292" and corresponds with the "HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES".  This means that in the "class_size" dataframe, the "CSD" should be "1" and the "school code" should be "M292" for that same school, which is confirmed in the next code cell. 

In [5]:
# View first 5 rows in "class_size" where "school code" is M292
data["class_size"][data["class_size"]["SCHOOL CODE"] == "M292"][["CSD", "SCHOOL CODE", "SCHOOL NAME"]].head()

,CSD,SCHOOL CODE,SCHOOL NAME
207,1,M292,Henry Street School for International Studies
208,1,M292,Henry Street School for International Studies
209,1,M292,Henry Street School for International Studies
210,1,M292,Henry Street School for International Studies
211,1,M292,Henry Street School for International Studies


### Exploring the "demographics" data  

Referencing the "Demographics_Data_Dictionary" in the "references" folder of this project, this dataset provides various demographic data for each NYC high school.  Some examples include: "percentage of English language learners" (ell_percent), "percentage of asian students" (asian_per) and "percentage of female students" (female_per).  This data will be instrumental in exploring if the SAT is unfair towards any particular demographic groups.

Each row represents a specific school year for that school.  We will have to filter for the rows that contain the school years that are most suitable for our analysis.  We can use the DBN column to join this data with the other dataframes.

In [6]:
# View the first 5 rows of the "demographics" dataframe
data["demographics"].head()

,DBN,Name,schoolyear,fl_percent,frl_percent,total_enrollment,prek,k,grade1,grade2,grade3,grade4,grade5,grade6,grade7,grade8,grade9,grade10,grade11,grade12,ell_num,ell_percent,sped_num,sped_percent,ctt_num,selfcontained_num,asian_num,asian_per,black_num,black_per,hispanic_num,hispanic_per,white_num,white_per,male_num,male_per,female_num,female_per
0,01M015,P.S. 015 ROBERTO CLEMENTE,20052006,89.4,NaN,281,15,36,40,33,38,52,29,38,NaN,NaN,NaN,NaN,NaN,NaN,36.0,12.8,57.0,20.3,25,9,10,3.6,74,26.3,189,67.3,5,1.8,158.0,56.2,123.0,43.8
1,01M015,P.S. 015 ROBERTO CLEMENTE,20062007,89.4,NaN,243,15,29,39,38,34,42,46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.0,15.6,55.0,22.6,19,15,18,7.4,68,28.0,153,63.0,4,1.6,140.0,57.6,103.0,42.4
2,01M015,P.S. 015 ROBERTO CLEMENTE,20072008,89.4,NaN,261,18,43,39,36,38,47,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.0,19.9,60.0,23.0,20,14,16,6.1,77,29.5,157,60.2,7,2.7,143.0,54.8,118.0,45.2
3,01M015,P.S. 015 ROBERTO CLEMENTE,20082009,89.4,NaN,252,17,37,44,32,34,39,49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.0,19.0,62.0,24.6,21,17,16,6.3,75,29.8,149,59.1,7,2.8,149.0,59.1,103.0,40.9
4,01M015,P.S. 015 ROBERTO CLEMENTE,20092010,,96.5,208,16,40,28,32,30,24,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,19.2,46.0,22.1,14,14,16,7.7,67,32.2,118,56.7,6,2.9,124.0,59.6,84.0,40.4


### Exploring the "graduation" data  

Referencing the "Graduation_Data_Dictionary" in the "references" folder of this project, this dataset provides various graduation statistics for each NYC high school.  Some examples include: "percentage of students that graduated" (Total Grads - % of cohort) and "percentage of students that dropped out" (Dropped Out - % of Cohort).  We can use this data to find any correlation between SAT results and graduation rates.

Each row represents a specific "demographic" (i.e. Total Cohort, White, Black etc...) and "cohort" (the school year the SAT was administered) for that school.  We will have to filter for the rows that contain the demographic and cohort that is most suitable for our analysis.  We can use the DBN column to join this data with the other dataframes.

In [7]:
# View the first 5 rows of the "graduation" dataframe
data["graduation"].head()

,Demographic,DBN,School Name,Cohort,Total Cohort,Total Grads - n,Total Grads - % of cohort,Total Regents - n,Total Regents - % of cohort,Total Regents - % of grads,Advanced Regents - n,Advanced Regents - % of cohort,Advanced Regents - % of grads,Regents w/o Advanced - n,Regents w/o Advanced - % of cohort,Regents w/o Advanced - % of grads,Local - n,Local - % of cohort,Local - % of grads,Still Enrolled - n,Still Enrolled - % of cohort,Dropped Out - n,Dropped Out - % of cohort
0,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2003,5,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s
1,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2004,55,37,67.3%,17,30.9%,45.9%,0,0%,0%,17,30.9%,45.9%,20,36.4%,54.1%,15,27.3%,3,5.5%
2,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2005,64,43,67.2%,27,42.2%,62.8%,0,0%,0%,27,42.2%,62.8%,16,25%,37.200000000000003%,9,14.1%,9,14.1%
3,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2006,78,43,55.1%,36,46.2%,83.7%,0,0%,0%,36,46.2%,83.7%,7,9%,16.3%,16,20.5%,11,14.1%
4,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2006 Aug,78,44,56.4%,37,47.4%,84.1%,0,0%,0%,37,47.4%,84.1%,7,9%,15.9%,15,19.2%,11,14.1%


### Exploring the "hs_directory" data  

Referencing the "HS_Directory_Data_Dictionary" in the "references" folder of this project, this dataset provides various descriptive information for each NYC high school.  Some examples include: "borough where the school is located" (boro) and "address/geographic coordinates" (Location 1).  We can use the location data to visualize the SAT results on a map of NYC.  This will allow us to easily see if there is any correlation between SAT results and specific areas of NYC.

Each row represents descriptive information for each NYC high school.  We can use the DBN column to join this data with the other dataframes.

In [8]:
# View the first 5 rows of the "hs_directory" dataframe
data["hs_directory"].head()

,dbn,school_name,boro,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,bus,subway,primary_address_line_1,city,state_code,zip,website,total_students,campus_name,school_type,overview_paragraph,program_highlights,language_classes,advancedplacement_courses,online_ap_courses,online_language_courses,extracurricular_activities,psal_sports_boys,psal_sports_girls,psal_sports_coed,school_sports,partner_cbo,partner_hospital,partner_highered,partner_cultural,partner_nonprofit,partner_corporate,partner_financial,partner_other,addtl_info1,addtl_info2,start_time,end_time,se_services,ell_programs,school_accessibility_description,number_programs,priority01,priority02,priority03,priority04,priority05,priority06,priority07,priority08,priority09,priority10,Location 1
0,17K548,Brooklyn School for Music & Theatre,Brooklyn,K440,718-230-6250,718-230-6262,9,12,NaN,NaN,"B41, B43, B44-SBS, B45, B48, B49, B69","2, 3, 4, 5, F, S to Botanic Garden ; B, Q to P...",883 Classon Avenue,Brooklyn,NY,11225,Bkmusicntheatre.com,399.0,Prospect Heights Educational Campus,NaN,Brooklyn School for Music & Theatre (BSMT) use...,We offer highly competitive positions in our D...,Spanish,"English Language and Composition, United State...",NaN,NaN,"Variety of clubs: Chess, The Step Team, Fashio...","Baseball, Basketball & JV Basketball, Cross Co...","Basketball, Cross Country, Indoor Track, Outdo...",NaN,NaN,F.Y.R.EZONE (Finding Your Rhythm thru Educatio...,NaN,NaN,"In 2002, Roundabout Theatre was selected by Ne...",One To World‘s Global Classroom connects New Y...,NaN,NaN,NaN,NaN,NaN,8:10 AM,3:00 PM,This school will provide students with disabil...,ESL,Functionally Accessible,1,Priority to Brooklyn students or residents,Then to New York City residents,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"883 Classon Avenue\nBrooklyn, NY 11225\n(40.67..."
1,09X543,High School for Violin and Dance,Bronx,X400,718-842-0687,718-589-9849,9,12,NaN,NaN,"Bx13, Bx15, Bx17, Bx21, Bx35, Bx4, Bx41, Bx4A,...","2, 5 to Intervale Ave",1110 Boston Road,Bronx,NY,10456,www.hsvd.org,378.0,Morris Educational Campus,NaN,The High School for Violin and Dance (HSVD) is...,Freshmen take both violin and dance; College N...,Spanish,NaN,NaN,NaN,Advancement via Individual Determination (AVID...,"Baseball, Basketball & JV Basketball, Volleyball","Basketball, Softball, Volleyball",NaN,Morris Educational Campus Basketball and Volle...,McGraw Hill - Big Brother Big Sister,NaN,"Hostos Community College, Monroe College, Teac...",Bronx Arts Ensemble,buildOn,Print International,NaN,Bronx Cares,Our students are required to take four years o...,"Student Summer Orientation, Summer Internship ...",8:00 AM,3:00 PM,This school will provide students with disabil...,ESL,Functionally Accessible,1,Priority to Bronx students or residents who at...,Then to New York City residents who attend an ...,Then to Bronx students or residents,Then to New York City residents,NaN,NaN,NaN,NaN,NaN,NaN,"1110 Boston Road\nBronx, NY 10456\n(40.8276026..."
2,09X327,Comprehensive Model School Project M.S. 327,Bronx,X240,718-294-8111,718-294-8109,6,12,NaN,NaN,"Bx1, Bx11, Bx13, Bx18, Bx2, Bx3, Bx32, Bx35, Bx36","4 to Mt Eden Ave ; B, D to 170th St",1501 Jerome Avenue,Bronx,NY,10452,http://schools.nyc.gov/schoolportals/09/X327,543.0,DOE New Settlement Community Campus,NaN,At the Comprehensive Model School Project (CMS...,"After-school and Saturday Tutoring, Advisory, ...",Spanish,"Biology, Chemistry, United States History",NaN,NaN,"Choir, Gaming, Girls Club, Newspaper, Spanish,...",NaN,NaN,NaN,"As we expand, we plan to offer PSAL sports.",New Settlement Community Center,Montefiore Hospital,NaN,NaN,NaN,NaN,NaN,NaN,Dress Code Required: white or baby blue button...,NaN,8:00 AM,4:00 PM,This school will provide students with disabil...,ESL,Functionally Accessible,1,Priority to continuing 8th graders,Then to Bronx students or residents who attend...,Then to New York City residents who attend an ...,Then to Bronx stud

# Reading in, exploring and combining the survey data  

Referencing [NYC OpenData](https://data.cityofnewyork.us/Education/2010-2011-NYC-School-Survey/mnz3-dyi8) and the "Survey_Data_Dictionary" in the "references" folder of this project, the "survey_all.txt" and "survey_d75.txt" files are datasets summarizing the results of the 2011 NYC School Survey.  This is an annual survey that asks parents, teachers and students their opinions on academic expectations, communication, engagement, safety, respect and various other topics at their respective schools.  


### Reading in and exploring the "survey_all.txt" data  

The "survey_all.txt" data file provides the survey results for all community schools.  We will read it into a dataframe named "all_survey".  

Some of the columns include: "safety and respect score based on student responses" (saf_s_11) and "academic expectations score based on teacher responses" (aca_t_11).  We can use this data to find any correlation between SAT results and parents', teachers' and students' opinions about their respective schools.

Each row represents the survey results for each NYC high school.  We can use the DBN column to join this data with the other dataframes.

In [9]:
# Read "survey_all.txt" into a dataframe and view the first 5 rows
all_survey = pd.read_csv('../data/raw/survey_all.txt', delimiter='\t', encoding='windows-1252')
all_survey.head()

,dbn,bn,schoolname,d75,studentssurveyed,highschool,schooltype,rr_s,rr_t,rr_p,N_s,N_t,N_p,nr_s,nr_t,nr_p,saf_p_11,com_p_11,eng_p_11,aca_p_11,saf_t_11,com_t_11,eng_t_11,aca_t_11,saf_s_11,com_s_11,eng_s_11,aca_s_11,saf_tot_11,com_tot_11,eng_tot_11,aca_tot_11,p_q2h,p_q7a,p_q7b,p_q7c,p_q7d,p_q8a,p_q8b,p_q8c,p_q8d,p_q8e,p_q8f,p_q2b,p_q2d,p_q2e,p_q2f,p_q2g,p_q3a,p_q3b,...,s_N_q13b_3,s_N_q13b_4,s_N_q13c_1,s_N_q13c_2,s_N_q13c_3,s_N_q13c_4,s_N_q13d_1,s_N_q13d_2,s_N_q13d_3,s_N_q13d_4,s_N_q13e_1,s_N_q13e_2,s_N_q13e_3,s_N_q13e_4,s_N_q13f_1,s_N_q13f_2,s_N_q13f_3,s_N_q13f_4,s_N_q13g_1,s_N_q13g_2,s_N_q13g_3,s_N_q13g_4,s_N_q14a_1,s_N_q14a_2,s_N_q14a_3,s_N_q14a_4,s_N_q14b_1,s_N_q14b_2,s_N_q14b_3,s_N_q14b_4,s_N_q14c_1,s_N_q14c_2,s_N_q14c_3,s_N_q14c_4,s_N_q14d_1,s_N_q14d_2,s_N_q14d_3,s_N_q14d_4,s_N_q14e_1,s_N_q14e_2,s_N_q14e_3,s_N_q14e_4,s_N_q14f_1,s_N_q14f_2,s_N_q14f_3,s_N_q14f_4,s_N_q14g_1,s_N_q14g_2,s_N_q14g_3,s_N_q14g_4
0,01M015,M015,P.S. 015 Roberto Clemente,0,No,0.0,Elementary School,NaN,88,60,NaN,22.0,90.0,0,25,150,8.5,7.6,7.5,7.8,7.5,7.8,7.6,7.9,NaN,NaN,NaN,NaN,8.0,7.7,7.5,7.9,8.0,8.2,8.3,7.5,7.9,6.8,8.7,9.7,8.7,9.9,9.9,7.7,8.3,7.9,8.1,7.5,7.3,6.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01M019,M019,P.S. 019 Asher Levy,0,No,0.0,Elementary School,NaN,100,60,NaN,34.0,161.0,0,33,269,8.4,7.6,7.6,7.8,8.6,8.5,8.9,9.1,NaN,NaN,NaN,NaN,8.5,8.1,8.2,8.4,7.7,7.9,8.0,7.3,7.7,6.5,8.8,9.4,8.7,10.0,9.9,7.5,8.2,7.9,8.0,7.2,7.0,6.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01M020,M020,P.S. 020 Anna Silver,0,No,0.0,Elementary School,NaN,88,73,NaN,42.0,367.0,0,48,505,8.9,8.3,8.3,8.6,7.6,6.3,6.8,7.5,NaN,NaN,NaN,NaN,8.2,7.3,7.5,8.0,8.1,8.8,8.9,8.5,8.4,7.6,9.2,9.4,9.2,9.8,9.7,8.4,8.8,8.5,8.3,8.0,7.6,7.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01M034,M034,P.S. 034 Franklin D. Roosevelt,0,Yes,0.0,Elementary / Middle School,89.0,73,50,145.0,29.0,151.0,163,40,301,8.8,8.2,8.0,8.5,7.0,6.2,6.8,7.8,6.2,5.9,6.5,7.4,7.3,6.7,7.1,7.9,8.1,8.5,8.8,8.2,8.3,7.3,9.2,9.4,9.1,9.8,9.7,8.3,8.7,8.3,8.1,7.6,7.6,7.5,...,19.0,16.0,24.0,74.0,20.0,24.0,21.0,50.0,26.0,45.0,76.0,35.0,14.0,17.0,100.0,30.0,6.0,4.0,89.0,28.0,9.0,12.0,32.0,68.0,21.0,16.0,23.0,60.0,32.0,25.0,43.0,66.0,19.0,12.0,39.0,68.0,18.0,16.0,35.0,70.0,20.0,16.0,23.0,54.0,33.0,29.0,31.0,46.0,16.0,8.0
4,01M063,M063,P.S. 063 William McKinley,0,No,0.0,Elementary School,NaN,100,60,NaN,23.0,90.0,0,23,151,8.7,7.9,8.1,7.9,8.4,7.3,7.8,8.1,NaN,NaN,NaN,NaN,8.5,7.6,7.9,8.0,8.0,8.4,8.6,8.0,8.0,6.5,8.8,9.6,9.4,10.0,10.0,7.6,8.5,8.0,8.2,7.4,7.8,7.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Reading in and exploring the "survey_d75.txt" data  

The "survey_d75.txt" data file provides the survey results for all District 75 schools.  We will read it into a dataframe named "d75_survey".  

The structure of the dataframe is essentially the same as "all_survey".  Although there are less total columns, the primary ones concering parents', teachers' and students' opinions about school engagement, safety, academic expectations etc... remain common in both dataframes.

In [10]:
# Read "survey_d75.txt" into a dataframe and view the first 5 rows
d75_survey = pd.read_csv('../data/raw/survey_d75.txt', delimiter='\t', encoding='windows-1252')
d75_survey.head()

,dbn,bn,schoolname,d75,studentssurveyed,highschool,schooltype,rr_s,rr_t,rr_p,N_s,N_t,N_p,nr_s,nr_t,nr_p,saf_p_11,com_p_11,eng_p_11,aca_p_11,saf_t_11,com_t_11,eng_t_11,aca_t_11,saf_s_11,com_s_11,eng_s_11,aca_s_11,saf_tot_11,com_tot_11,eng_tot_11,aca_tot_11,p_q1c,p_q10a,p_q10b,p_q10c,p_q10d,p_q10e,p_q10f,p_q11a,p_q11b,p_q11c,p_q11d,p_q11e,p_q1b,p_q1e,p_q1f,p_q2a,p_q2b,p_q3c,...,s_q12e_2,s_q12e_3,s_q12e_4,s_q12f_1,s_q12f_2,s_q12f_3,s_q12f_4,s_q12g_1,s_q12g_2,s_q12g_3,s_q12g_4,s_q13a_1,s_q13a_2,s_q13a_3,s_q13a_4,s_q13b_1,s_q13b_2,s_q13b_3,s_q13b_4,s_q13c_1,s_q13c_2,s_q13c_3,s_q13c_4,s_q13d_1,s_q13d_2,s_q13d_3,s_q13d_4,s_q13e_1,s_q13e_2,s_q13e_3,s_q13e_4,s_q13f_1,s_q13f_2,s_q13f_3,s_q13f_4,s_q13g_1,s_q13g_2,s_q13g_3,s_q13g_4,s_q14_1,s_q14_2,s_q14_3,s_q14_4,s_q14_5,s_q14_6,s_q14_7,s_q14_8,s_q14_9,s_q14_10,s_q14_11
0,75K004,K004,P.S. K004,1,Yes,0.0,District 75 Special Education,38.0,90,72,8.0,81,244,21,90,337,9.1,8.6,8.4,8.5,7.4,6.6,6.4,6.4,7.6,5.8,7.6,6.3,8.0,7.0,7.5,7.1,8.7,9.0,9.0,8.9,8.8,8.9,8.9,7.3,9.7,9.7,9.9,9.9,8.6,8.4,9.0,9.0,8.6,8.0,...,0.0,0.0,0.0,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,14.0,86.0,0.0,0.0,14.0,86.0,0.0,0.0,0.0,86.0,14.0,0.0,0.0,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,100.0,0.0,0.0,71.0,29.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,75K036,K036,P.S. 36,1,Yes,NaN,District 75 Special Education,70.0,69,44,97.0,43,115,139,62,263,9.2,8.7,8.6,8.8,6.9,7.1,6.1,7.2,6.6,6.9,7.3,7.7,7.5,7.6,7.3,7.9,8.8,9.3,9.2,9.0,9.1,9.3,9.2,7.6,9.7,8.8,10.0,9.8,9.2,9.0,9.1,8.4,8.0,8.5,...,32.0,13.0,10.0,68.0,20.0,4.0,8.0,54.0,22.0,13.0,10.0,46.0,43.0,7.0,4.0,42.0,39.0,17.0,2.0,52.0,39.0,4.0,4.0,41.0,40.0,12.0,7.0,42.0,31.0,19.0,8.0,37.0,38.0,12.0,12.0,35.0,47.0,10.0,8.0,13.0,20.0,27.0,19.0,9.0,2.0,6.0,1.0,2.0,0.0,0.0
2,75K053,K053,P.S. K053,1,Yes,NaN,District 75 Special Education,94.0,97,53,131.0,75,189,139,77,355,8.8,8.1,8.0,8.1,7.8,7.5,6.9,8.0,8.2,7.7,8.6,8.5,8.2,7.8,7.8,8.2,8.3,8.6,8.8,8.4,8.6,8.7,8.4,7.3,9.6,9.0,9.7,9.7,8.1,7.9,8.5,7.8,7.3,7.9,...,14.0,3.0,3.0,96.0,4.0,0.0,0.0,89.0,10.0,0.0,1.0,52.0,39.0,8.0,1.0,53.0,40.0,5.0,2.0,70.0,26.0,4.0,0.0,66.0,29.0,4.0,1.0,63.0,35.0,2.0,0.0,50.0,33.0,13.0,4.0,58.0,41.0,1.0,1.0,5.0,14.0,12.0,12.0,10.0,21.0,13.0,11.0,2.0,0.0,0.0
3,75K077,K077,P.S. K077,1,Yes,NaN,District 75 Special Education,95.0,65,55,70.0,52,150,74,80,274,9.4,8.7,8.8,8.9,7.1,6.2,6.4,6.3,8.6,8.3,8.6,8.8,8.3,7.7,7.9,8.0,8.8,9.2,9.2,9.1,9.1,9.1,8.9,9.5,10.0,9.9,10.0,10.0,9.0,8.7,8.9,8.8,8.0,8.7,...,7.0,0.0,3.0,96.0,1.0,3.0,0.0,97.0,0.0,0.0,3.0,54.0,43.0,1.0,1.0,55.0,36.0,7.0,1.0,59.0,38.0,1.0,1.0,57.0,35.0,4.0,4.0,61.0,36.0,1.0,1.0,56.0,34.0,7.0,3.0,64.0,33.0,1.0,1.0,3.0,14.0,14.0,7.0,11.0,16.0,10.0,6.0,4.0,7.0,7.0
4,75K140,K140,P.S. K140,1,Yes,0.0,District 75 Special Education,77.0,70,42,69.0,45,117,90,64,277,8.6,8.2,7.9,8.3,5.6,5.8,5.1,6.0,4.7,6.1,6.0,7.3,6.3,6.7,6.3,7.2,8.8,8.6,8.8,8.4,8.4,8.5,8.6,6.4,9.1,8.5,9.9,9.1,8.8,8.6,8.8,7.7,7.1,8.0,...,22.0,17.0,39.0,33.0,48.0,10.0,10.0,16.0,38.0,11.0,34.0,18.0,62.0,5.0,15.0,23.0,27.0,17.0,33.0,34.0,25.0,31.0,11.0,17.0,14.0,42.0,28.0,18.0,11.0,34.0,37.0,6.0,22.0,25.0,48.0,19.0,25.0,22.0,34.0,12.0,35.0,34.0,17.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0


### Combining the survey dataframes  

There is no need for us to separate the survey results for District 75 schools, so we will join the "d75_survey" dataframe to the bottom of the "all_survey" dataframe and create one final dataframe named "survey".  

We will also view the first and last rows to confirm that the new dataframe contains both Non-District 75 schools (DBN does not begin with "75") and District 75 schools (DBN does begin with "75").

In [11]:
# Join "d75_survey" to the bottom of "all_survey" and create new dataframe "survey"
survey = pd.concat([all_survey, d75_survey], axis=0, sort=False)

# View the first and last rows of "survey"
survey.iloc[[1,-1]]

,dbn,bn,schoolname,d75,studentssurveyed,highschool,schooltype,rr_s,rr_t,rr_p,N_s,N_t,N_p,nr_s,nr_t,nr_p,saf_p_11,com_p_11,eng_p_11,aca_p_11,saf_t_11,com_t_11,eng_t_11,aca_t_11,saf_s_11,com_s_11,eng_s_11,aca_s_11,saf_tot_11,com_tot_11,eng_tot_11,aca_tot_11,p_q2h,p_q7a,p_q7b,p_q7c,p_q7d,p_q8a,p_q8b,p_q8c,p_q8d,p_q8e,p_q8f,p_q2b,p_q2d,p_q2e,p_q2f,p_q2g,p_q3a,p_q3b,...,s_q9j_3,s_q9k_1,s_q9k_2,s_q9k_3,s_q9l_1,s_q9l_2,s_q9l_3,s_q10_1,s_q10_2,s_q10_3,s_q10_4,s_q11a_1,s_q11a_2,s_q11a_3,s_q11a_4,s_q11b_1,s_q11b_2,s_q11b_3,s_q11b_4,s_q11c_1,s_q11c_2,s_q11c_3,s_q11c_4,s_q12d_1,s_q12d_2,s_q12d_3,s_q12d_4,s_q12e_1,s_q12e_2,s_q12e_3,s_q12e_4,s_q12f_1,s_q12f_2,s_q12f_3,s_q12f_4,s_q12g_1,s_q12g_2,s_q12g_3,s_q12g_4,s_q14_1,s_q14_2,s_q14_3,s_q14_4,s_q14_5,s_q14_6,s_q14_7,s_q14_8,s_q14_9,s_q14_10,s_q14_11
1,01M019,M019,P.S. 019 Asher Levy,0,No,0.0,Elementary School,NaN,100,60,NaN,34.0,161.0,0,33,269,8.4,7.6,7.6,7.8,8.6,8.5,8.9,9.1,NaN,NaN,NaN,NaN,8.5,8.1,8.2,8.4,7.7,7.9,8.0,7.3,7.7,6.5,8.8,9.4,8.7,10.0,9.9,7.5,8.2,7.9,8.0,7.2,7.0,6.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55,75X811,X811,P.S. X811,1,Yes,NaN,District 75 Special Education,38.0,63,19,44.0,71.0,119.0,115,113,627,8.5,7.8,7.7,7.8,6.9,6.6,6.5,7.0,7.7,6.1,7.7,7.7,7.7,6.8,7.3,7.5,NaN,53.0,48.0,27.0,11.0,7.9,7.6,NaN,NaN,NaN,NaN,6.9,NaN,NaN,NaN,NaN,7.8,7.3,...,42.0,43.0,22.0,35.0,66.0,8.0,26.0,57.0,29.0,10.0,5.0,33.0,43.0,14.0,10.0,22.0,49.0,20.0,10.0,44.0,32.0,20.0,5.0,66.0,24.0,2.0,7.0,73.0,20.0,5.0,2.0,93.0,5.0,0.0,2.0,90.0,5.0,0.0,5.0,0.0,2.0,0.0,0.0,7.0,7.0,19.0,29.0,17.0,12.0,7.0



# Cleaning the "survey" dataframe  

In the "survey" dataframe, we will copy the values of the "dbn" column into a new "DBN" column (all capital letters) so it is consistent with the other dataframes.  

There are 2773 columns in this dataframe but we will only need a select few for our analysis.  Based on reviewing the "Survey_Data_Dictionary" and hypothesizing which data could possibly influence SAT results, we will only keep the following columns:  

* DBN: School identification code (district, borough, school number)  


* rr_s: Student response rate  
* rr_t: Teacher response rate  
* rr_p: Parent response rate  


* N_s: Number of student respondents  
* N_t: Number of teacher respondents  
* N_p: Number of parent respondents  


* saf_s_11: Safety and respect score based on student responses  
* saf_t_11: Safety and respect score based on teacher responses
* saf_p_11: Safety and respect score based on parent responses  
* saf_tot_11: Safety and respect total score  


* eng_s_11: Engagement score based on student responses  
* eng_t_11: Engagement score based on teacher responses  
* eng_p_11: Engagement score based on parent responses
* eng_tot_11: Engagement total score  


* aca_s_11: Academic expectations score based on student responses  
* aca_t_11: Academic expectations score based on teacher responses  
* aca_p_11: Academic expectations score based on parent responses
* aca_tot_11: Academic expectations total score


* com_s_11: Communication score based on student responses  
* com_t_11: Communication score based on teacher responses
* com_p_11: Communication score based on parent responses
* com_tot_11: Communication total score  

After filtering "survey" to only include the above selected columns, we will add it to our "data" dictionary along with the other dataframes.

In [12]:
# Create a list of our selected column names
survey_cols = [
    "DBN", "rr_s", "rr_t", "rr_p", "N_s", "N_t", "N_p",
    "saf_s_11", "saf_t_11", "saf_p_11", "saf_tot_11",
    "eng_s_11", "eng_t_11", "eng_p_11", "eng_tot_11",
    "aca_s_11", "aca_t_11", "aca_p_11", "aca_tot_11",
    "com_s_11", "com_t_11", "com_p_11", "com_tot_11"
]

# Create the "DBN" column and filter "survey" to only include our selected columns
survey["DBN"] = survey["dbn"]
survey = survey.loc[:, survey_cols]

# Add "survey" to our "data" dictionary and view the first 5 rows
data["survey"] = survey
data["survey"].head()

,DBN,rr_s,rr_t,rr_p,N_s,N_t,N_p,saf_s_11,saf_t_11,saf_p_11,saf_tot_11,eng_s_11,eng_t_11,eng_p_11,eng_tot_11,aca_s_11,aca_t_11,aca_p_11,aca_tot_11,com_s_11,com_t_11,com_p_11,com_tot_11
0,01M015,NaN,88,60,NaN,22.0,90.0,NaN,7.5,8.5,8.0,NaN,7.6,7.5,7.5,NaN,7.9,7.8,7.9,NaN,7.8,7.6,7.7
1,01M019,NaN,100,60,NaN,34.0,161.0,NaN,8.6,8.4,8.5,NaN,8.9,7.6,8.2,NaN,9.1,7.8,8.4,NaN,8.5,7.6,8.1
2,01M020,NaN,88,73,NaN,42.0,367.0,NaN,7.6,8.9,8.2,NaN,6.8,8.3,7.5,NaN,7.5,8.6,8.0,NaN,6.3,8.3,7.3
3,01M034,89.0,73,50,145.0,29.0,151.0,6.2,7.0,8.8,7.3,6.5,6.8,8.0,7.1,7.4,7.8,8.5,7.9,5.9,6.2,8.2,6.7
4,01M063,NaN,100,60,NaN,23.0,90.0,NaN,8.4,8.7,8.5,NaN,7.8,8.1,7.9,NaN,8.1,7.9,8.0,NaN,7.3,7.9,7.6


# Inserting DBN columns  

### Inserting the DBN column into "hs_directory"

In the "hs_directory" dataframe, we will copy the values of the "dbn" column into a new "DBN" column (all capital letters) so it is consistent with the other dataframes.  We then view the first 5 rows to confirm it was created properly.  The new "DBN" column will be at the right most end of the dataframe.

In [13]:
# Create the "DBN" column and view the first 5 rows
data["hs_directory"]["DBN"] = data["hs_directory"]["dbn"]
data["hs_directory"].head()

,dbn,school_name,boro,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,bus,subway,primary_address_line_1,city,state_code,zip,website,total_students,campus_name,school_type,overview_paragraph,program_highlights,language_classes,advancedplacement_courses,online_ap_courses,online_language_courses,extracurricular_activities,psal_sports_boys,psal_sports_girls,psal_sports_coed,school_sports,partner_cbo,partner_hospital,partner_highered,partner_cultural,partner_nonprofit,partner_corporate,partner_financial,partner_other,addtl_info1,addtl_info2,start_time,end_time,se_services,ell_programs,school_accessibility_description,number_programs,priority01,priority02,priority03,priority04,priority05,priority06,priority07,priority08,priority09,priority10,Location 1,DBN
0,17K548,Brooklyn School for Music & Theatre,Brooklyn,K440,718-230-6250,718-230-6262,9,12,NaN,NaN,"B41, B43, B44-SBS, B45, B48, B49, B69","2, 3, 4, 5, F, S to Botanic Garden ; B, Q to P...",883 Classon Avenue,Brooklyn,NY,11225,Bkmusicntheatre.com,399.0,Prospect Heights Educational Campus,NaN,Brooklyn School for Music & Theatre (BSMT) use...,We offer highly competitive positions in our D...,Spanish,"English Language and Composition, United State...",NaN,NaN,"Variety of clubs: Chess, The Step Team, Fashio...","Baseball, Basketball & JV Basketball, Cross Co...","Basketball, Cross Country, Indoor Track, Outdo...",NaN,NaN,F.Y.R.EZONE (Finding Your Rhythm thru Educatio...,NaN,NaN,"In 2002, Roundabout Theatre was selected by Ne...",One To World‘s Global Classroom connects New Y...,NaN,NaN,NaN,NaN,NaN,8:10 AM,3:00 PM,This school will provide students with disabil...,ESL,Functionally Accessible,1,Priority to Brooklyn students or residents,Then to New York City residents,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"883 Classon Avenue\nBrooklyn, NY 11225\n(40.67...",17K548
1,09X543,High School for Violin and Dance,Bronx,X400,718-842-0687,718-589-9849,9,12,NaN,NaN,"Bx13, Bx15, Bx17, Bx21, Bx35, Bx4, Bx41, Bx4A,...","2, 5 to Intervale Ave",1110 Boston Road,Bronx,NY,10456,www.hsvd.org,378.0,Morris Educational Campus,NaN,The High School for Violin and Dance (HSVD) is...,Freshmen take both violin and dance; College N...,Spanish,NaN,NaN,NaN,Advancement via Individual Determination (AVID...,"Baseball, Basketball & JV Basketball, Volleyball","Basketball, Softball, Volleyball",NaN,Morris Educational Campus Basketball and Volle...,McGraw Hill - Big Brother Big Sister,NaN,"Hostos Community College, Monroe College, Teac...",Bronx Arts Ensemble,buildOn,Print International,NaN,Bronx Cares,Our students are required to take four years o...,"Student Summer Orientation, Summer Internship ...",8:00 AM,3:00 PM,This school will provide students with disabil...,ESL,Functionally Accessible,1,Priority to Bronx students or residents who at...,Then to New York City residents who attend an ...,Then to Bronx students or residents,Then to New York City residents,NaN,NaN,NaN,NaN,NaN,NaN,"1110 Boston Road\nBronx, NY 10456\n(40.8276026...",09X543
2,09X327,Comprehensive Model School Project M.S. 327,Bronx,X240,718-294-8111,718-294-8109,6,12,NaN,NaN,"Bx1, Bx11, Bx13, Bx18, Bx2, Bx3, Bx32, Bx35, Bx36","4 to Mt Eden Ave ; B, D to 170th St",1501 Jerome Avenue,Bronx,NY,10452,http://schools.nyc.gov/schoolportals/09/X327,543.0,DOE New Settlement Community Campus,NaN,At the Comprehensive Model School Project (CMS...,"After-school and Saturday Tutoring, Advisory, ...",Spanish,"Biology, Chemistry, United States History",NaN,NaN,"Choir, Gaming, Girls Club, Newspaper, Spanish,...",NaN,NaN,NaN,"As we expand, we plan to offer PSAL sports.",New Settlement Community Center,Montefiore Hospital,NaN,NaN,NaN,NaN,NaN,NaN,Dress Code Required: white or baby blue button...,NaN,8:00 AM,4:00 PM,This school will provide students with disabil...,ESL,Functionally Accessible,1,Priority to continuing 8th graders,Then to Bronx students or residents who attend...,Then to New York City residents who attend an ...,

### Inserting the DBN column into "class_size"  

As we discovered previously, "class_size" doesn't have a "DBN" column but it can be created by joining the "CSD" and "school code" columns.  If "CSD" is only a single digit, we need to add a leading "0" to make the DBN consistent with the other dataframes.  

We will create a function named "pad_csd" which takes in an integer value, converts it to a string, adds a leading "0" if it is only a single digit and returns the string.  We will then apply this function to the "CSD" column to create a new column named "padded_csd", which will contain all of the "CSD" values as 2-digit integer strings.  Finally, we will combine the "padded_csd" and "school code" columns to create the "DBN" column.  

We will then view the first 5 rows to confirm both columns were created properly. The new "padded_csd" and "DBN" columns will be at the right most end of the dataframe.

In [14]:
def pad_csd(int):
    """Convert an integer to a string, add a leading '0' if it's a single digit and return the string."""
    string = str(int)
    if len(string) == 2:
        return string
    else:
        return string.zfill(2)

# Create "padded_csd" and "DBN" columns and view first 5 rows
data["class_size"]["padded_csd"] = data["class_size"]["CSD"].apply(pad_csd)
data["class_size"]["DBN"] = data["class_size"]["padded_csd"] + data["class_size"]["SCHOOL CODE"]
data["class_size"].head()

,CSD,BOROUGH,SCHOOL CODE,SCHOOL NAME,GRADE,PROGRAM TYPE,CORE SUBJECT (MS CORE and 9-12 ONLY),CORE COURSE (MS CORE and 9-12 ONLY),SERVICE CATEGORY(K-9* ONLY),NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,DATA SOURCE,SCHOOLWIDE PUPIL-TEACHER RATIO,padded_csd,DBN
0,1,M,M015,P.S. 015 Roberto Clemente,0K,GEN ED,-,-,-,19.0,1.0,19.0,19.0,19.0,ATS,NaN,01,01M015
1,1,M,M015,P.S. 015 Roberto Clemente,0K,CTT,-,-,-,21.0,1.0,21.0,21.0,21.0,ATS,NaN,01,01M015
2,1,M,M015,P.S. 015 Roberto Clemente,01,GEN ED,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN,01,01M015
3,1,M,M015,P.S. 015 Roberto Clemente,01,CTT,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN,01,01M015
4,1,M,M015,P.S. 015 Roberto Clemente,02,GEN ED,-,-,-,15.0,1.0,15.0,15.0,15.0,ATS,NaN,01,01M015


# Combining the SAT scores  

We are only interested in the total SAT score for each school from "sat_results".  We can calculate the total SAT score by adding together the 3 categorical scores: critical reading, math and writing.  

First, we will convert those 3 columns from string to numeric data types so that arithmetic operations can be performed on them.  Then we will add them together to create a new column of total SAT scores named "sat_score".  Finally, we will view the first 5 rows to confirm the new column was created properly.

In [15]:
# Create a list of the columns from "sat_results" to convert
sat_cols = [
    "SAT Critical Reading Avg. Score",
    "SAT Math Avg. Score",
    "SAT Writing Avg. Score"
]

# Convert the selected columns to numeric data types
for col in sat_cols:
    data["sat_results"]["{0}".format(col)] = pd.to_numeric(
        data["sat_results"]["{0}".format(col)], errors="coerce")

# Create the "sat_score" column by adding together the converted columns
data["sat_results"]["sat_score"] = (data["sat_results"]["SAT Critical Reading Avg. Score"] 
                                    + data["sat_results"]["SAT Math Avg. Score"] 
                                    + data["sat_results"]["SAT Writing Avg. Score"])

# View the first 5 rows
data['sat_results'].head()

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,sat_score
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355.0,404.0,363.0,1122.0
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383.0,423.0,366.0,1172.0
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377.0,402.0,370.0,1149.0
3,01M458,FORSYTH SATELLITE ACADEMY,7,414.0,401.0,359.0,1174.0
4,01M509,MARTA VALLE HIGH SCHOOL,44,390.0,433.0,384.0,1207.0


# Parsing geographic coordinates for schools  

As discussed earlier, the "location 1" column of the "hs_directory" dataframe contains the geographic coordinates (latitude, longitude) of each school.  We are going to extract these latitudes and longitudes so that we can plot the SAT results of each school on a map of NYC.

### Extracting and converting the latitude  

In the "location 1" column, the geographic coordinates for each school are contained within parentheses "()".  The latitude (lat) is the first coordinate.  

We will import the "regular expression" (re) module and create a function named "get_lat".  This function will take in a string, find all of the characters that are contained within parentheses (the coordinates), select the first coordinate (the lat), drop the parentheses and return the cleaned lat value.  

We will then apply this function to the "location 1" column to extract all of the cleaned lat values and populate a new column named "lat".  We will convert this column from a string to a numeric data type so that arithmetic operations can be performed on it.  We will then view the first 5 rows of the "location 1" and "lat" columns to confirm the new column was created properly.

In [16]:
import re

def get_lat(string):
    """Find all coordinates inside parentheses in a string, select the lat, clean and return the lat."""
    coord = re.findall("\(.+\)", string)[0]
    lat = coord.split(", ")[0]
    lat_clean = lat.replace("(", "").replace(")", "")    
    return lat_clean

# Create the "lat" column, convert to numeric data type and view the first 5 rows of relevant columns
data["hs_directory"]["lat"] = data["hs_directory"]["Location 1"].apply(get_lat)
data["hs_directory"]["lat"] = pd.to_numeric(data["hs_directory"]["lat"], errors="coerce")
data["hs_directory"][["Location 1", "lat"]].head()

,Location 1,lat
0,"883 Classon Avenue\nBrooklyn, NY 11225\n(40.67...",40.670299
1,"1110 Boston Road\nBronx, NY 10456\n(40.8276026...",40.827603
2,"1501 Jerome Avenue\nBronx, NY 10452\n(40.84241...",40.842414
3,"411 Pearl Street\nNew York, NY 10038\n(40.7106...",40.710679
4,"160-20 Goethals Avenue\nJamaica, NY 11432\n(40...",40.718810


### Extracting and converting the longitude  

All of the steps here are analogous to the "extracting and converting the latitude" steps above, except we are extracting the 2nd coordinate, the longitude (lon), and creating the new column "lon".  

We will view the first 5 rows of the "location 1", "lat" and "lon" columns to confirm everything looks correct.

In [17]:
def get_lon(string):
    """Find all coordinates inside parentheses in a string, select the lon, clean and return the lon."""
    coord = re.findall("\(.+\)", string)[0]
    lon = coord.split(", ")[1]
    lon_clean = lon.replace("(", "").replace(")", "")    
    return lon_clean

# Create the "lon" column, convert to numeric data type and view the first 5 rows of relevant columns
data["hs_directory"]["lon"] = data["hs_directory"]["Location 1"].apply(get_lon)
data["hs_directory"]["lon"] = pd.to_numeric(data["hs_directory"]["lon"], errors="coerce")
data["hs_directory"][["Location 1", "lat", "lon"]].head()

,Location 1,lat,lon
0,"883 Classon Avenue\nBrooklyn, NY 11225\n(40.67...",40.670299,-73.961648
1,"1110 Boston Road\nBronx, NY 10456\n(40.8276026...",40.827603,-73.904475
2,"1501 Jerome Avenue\nBronx, NY 10452\n(40.84241...",40.842414,-73.916162
3,"411 Pearl Street\nNew York, NY 10038\n(40.7106...",40.710679,-74.000807
4,"160-20 Goethals Avenue\nJamaica, NY 11432\n(40...",40.718810,-73.806500


# Condensing the "class_size" dataframe  

As we discussed earlier when exploring the "class_size" data, each row of the dataframe represents a specific grade level and program type for that school.  We will have to filter for the rows that contain the grade levels and program types that are most suitable for our analysis.  

For ease and convenience we are first going to assign the "class_size" dataframe in our "data" dictionary to a variable named "class_size".  We will then view the unique values of the "grade" column.

In [18]:
# Assign the "class_size" dataframe from our "data" dictionary to a variable named "class_size"
# Then view the unique values in the "grade" column of "class_size"
class_size = data["class_size"]
list(class_size["GRADE "].unique())

['0K',
 '01',
 '02',
 '03',
 '04',
 '05',
 '0K-09',
 nan,
 '06',
 '07',
 '08',
 'MS Core',
 '09-12',
 '09']

Similarly, we will view the unique values of the "program type" column.

In [19]:
# View the unique values in the "program type" column of "class_size"
list(class_size["PROGRAM TYPE"].unique())

['GEN ED', 'CTT', 'SPEC ED', nan, 'G&T']

Since we are only exploring high school data, we will filter the dataframe to only include rows where the "grade" column is "09-12".  

Referencing the "Class_Size_Data_Dictionary" in the "references" folder of this project, the program types "CTT" (collaborative team teaching), "spec ed" (self-contained special education) and "G&T" (gifted and talented) are all specialty programs.  So we will filter the dataframe to only include the "gen ed" (general education) program type.  

We will then view the first 5 rows of "class_size" to confirm it's only including the rows we selected.

In [20]:
# Filter "class_size" to only include rows with our selected "grade" and "program type" values
class_size = class_size[class_size["GRADE "] == "09-12"]
class_size = class_size[class_size["PROGRAM TYPE"] == "GEN ED"]

# View the first 5 rows of "class_size"
class_size.head()

,CSD,BOROUGH,SCHOOL CODE,SCHOOL NAME,GRADE,PROGRAM TYPE,CORE SUBJECT (MS CORE and 9-12 ONLY),CORE COURSE (MS CORE and 9-12 ONLY),SERVICE CATEGORY(K-9* ONLY),NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,DATA SOURCE,SCHOOLWIDE PUPIL-TEACHER RATIO,padded_csd,DBN
225,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,ENGLISH,English 9,-,63.0,3.0,21.0,19.0,25.0,STARS,NaN,01,01M292
226,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,ENGLISH,English 10,-,79.0,3.0,26.3,24.0,31.0,STARS,NaN,01,01M292
227,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,ENGLISH,English 11,-,38.0,2.0,19.0,16.0,22.0,STARS,NaN,01,01M292
228,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,ENGLISH,English 12,-,69.0,3.0,23.0,13.0,30.0,STARS,NaN,01,01M292
229,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,MATH,Integrated Algebra,-,53.0,3.0,17.7,16.0,21.0,STARS,NaN,01,01M292


# Creating a unique "DBN" for each row in "class_size"  

Each row in "class_size" now represents a specific "core course" for that school (i.e. English 9, English 10, Integrated Algebra etc...).  This means there are still identical "DBN" values across multiple rows.  We need to aggregate the values in "class_size" so there is a unique "DBN" in each row.  

We will accomplish this by first importing the "numpy" module to use its "mean" function.  We will then group "class_size" by each unique "DBN" value and aggregate the values by calculating the mean of each column within each "DBN" group.  We will then reset the index of the new dataframe.  

We will assign the new "class_size" dataframe, with unique "DBN" values in each row, back to our "data" dictionary and view the first 5 rows.

In [21]:
import numpy as np

# Group "class_size" by "DBN", aggregate with the mean of the values and reset the index
class_size = class_size.groupby("DBN").agg(np.mean)
class_size.reset_index(inplace=True)

# Assign "class_size" back to our "data" dictionary and view the first 5 rows
data["class_size"] = class_size
data["class_size"].head()

,DBN,CSD,NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,SCHOOLWIDE PUPIL-TEACHER RATIO
0,01M292,1,88.0000,4.000000,22.564286,18.50,26.571429,NaN
1,01M332,1,46.0000,2.000000,22.000000,21.00,23.500000,NaN
2,01M378,1,33.0000,1.000000,33.000000,33.00,33.000000,NaN
3,01M448,1,105.6875,4.750000,22.231250,18.25,27.062500,NaN
4,01M450,1,57.6000,2.733333,21.200000,19.40,22.866667,NaN


# Condensing the "demographics" dataframe  

As we discussed earlier when exploring the "demographics" data, each row represents a specific school year for that school. We will have to filter for the rows that contain the school years that are most suitable for our analysis.  

First we will examine the unique values in the "schoolyear" column of "demographics".

In [22]:
# View the unique values in the "schoolyear" column of "demographics"
list(data["demographics"]["schoolyear"].unique())

[20052006, 20062007, 20072008, 20082009, 20092010, 20102011, 20112012]

We want the most recent data available, so we will filter the dataframe to only include the "schoolyear" value of "20112012".  This will also ensure each row has a unique "DBN" value.  We will then view the first 5 rows to confirm the dataframe has been condensed properly.

In [23]:
# Filter "demographics" to only include the "schoolyear" "20112012" and view the first 5 rows
data["demographics"] = data["demographics"][data["demographics"]["schoolyear"] == 20112012]
data["demographics"].head()

,DBN,Name,schoolyear,fl_percent,frl_percent,total_enrollment,prek,k,grade1,grade2,grade3,grade4,grade5,grade6,grade7,grade8,grade9,grade10,grade11,grade12,ell_num,ell_percent,sped_num,sped_percent,ctt_num,selfcontained_num,asian_num,asian_per,black_num,black_per,hispanic_num,hispanic_per,white_num,white_per,male_num,male_per,female_num,female_per
6,01M015,P.S. 015 ROBERTO CLEMENTE,20112012,NaN,89.4,189,13,31,35,28,25,28,29,,,,,,,,20.0,10.6,40.0,21.2,23,7,12,6.3,63,33.3,109,57.7,4,2.1,97.0,51.3,92.0,48.7
13,01M019,P.S. 019 ASHER LEVY,20112012,NaN,61.5,328,32,46,52,54,52,46,46,,,,,,,,33.0,10.1,59.0,18.0,16,16,51,15.5,81,24.7,158,48.2,28,8.5,147.0,44.8,181.0,55.2
20,01M020,PS 020 ANNA SILVER,20112012,NaN,92.5,626,52,102,121,87,88,85,91,,,,,,,,128.0,20.4,97.0,15.5,49,31,190,30.4,55,8.8,357,57.0,16,2.6,330.0,52.7,296.0,47.3
27,01M034,PS 034 FRANKLIN D ROOSEVELT,20112012,NaN,99.7,401,14,34,38,36,45,28,40,55,55,56,,,,,34.0,8.5,106.0,26.4,59,16,22,5.5,90,22.4,275,68.6,8,2.0,204.0,50.9,197.0,49.1
35,01M063,PS 063 WILLIAM MCKINLEY,20112012,NaN,78.9,176,18,20,30,21,31,26,30,,,,,,,,6.0,3.4,45.0,25.6,34,4,9,5.1,41,23.3,110,62.5,15,8.5,97.0,55.1,79.0,44.9


# Condensing the "graduation" dataframe  

As we discussed earlier when exploring the "graduation" data, each row represents a specific "demographic" and "cohort" for that school. We will have to filter for the rows that contain the demographic and cohort values that are most suitable for our analysis.

First we will examine the unique values in the "demographic" column of "graduation".

In [24]:
# View the unique values in the "demographic" column of "graduation"
list(data["graduation"]["Demographic"].unique())

['Total Cohort',
 'Male',
 'English Language Learners',
 'English Proficient Students',
 'Special Education Students',
 'General Education Students',
 'Asian',
 'Black',
 'Hispanic',
 'White',
 'Female']

Similarly, we will view the unique values in the "cohort" column.

In [25]:
# View the unique values in the "cohort" column of "graduation"
list(data["graduation"]["Cohort"].unique())

['2003', '2004', '2005', '2006', '2006 Aug', '2001', '2002']

We already have demographic specific data in the "demographics" dataframe, so we will filter the "graduation" dataframe to only include the "demographic" value of "Total Cohort".  

We want the most recent full-year data available, so we will filter the dataframe to only include the "cohort" value of "2006". 

Both of these filters will also ensure that each row has a unique "DBN" value.  We will then view the first 5 rows to confirm the dataframe has been condensed properly.

In [26]:
# Filter "graduation" to only include rows with our selected "demographic" and "cohort" values
data["graduation"] = data["graduation"][data["graduation"]["Demographic"] == "Total Cohort"]
data["graduation"] = data["graduation"][data["graduation"]["Cohort"] == "2006"]

# View the first 5 rows of "graduation"
data["graduation"].head()

,Demographic,DBN,School Name,Cohort,Total Cohort,Total Grads - n,Total Grads - % of cohort,Total Regents - n,Total Regents - % of cohort,Total Regents - % of grads,Advanced Regents - n,Advanced Regents - % of cohort,Advanced Regents - % of grads,Regents w/o Advanced - n,Regents w/o Advanced - % of cohort,Regents w/o Advanced - % of grads,Local - n,Local - % of cohort,Local - % of grads,Still Enrolled - n,Still Enrolled - % of cohort,Dropped Out - n,Dropped Out - % of cohort
3,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2006,78,43,55.1%,36,46.2%,83.7%,0,0%,0%,36,46.2%,83.7%,7,9%,16.3%,16,20.5%,11,14.1%
10,Total Cohort,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,2006,124,53,42.7%,42,33.9%,79.2%,8,6.5%,15.1%,34,27.4%,64.2%,11,8.9%,20.8%,46,37.1%,20,16.100000000000001%
17,Total Cohort,01M450,EAST SIDE COMMUNITY SCHOOL,2006,90,70,77.8%,67,74.400000000000006%,95.7%,0,0%,0%,67,74.400000000000006%,95.7%,3,3.3%,4.3%,15,16.7%,5,5.6%
24,Total Cohort,01M509,MARTA VALLE HIGH SCHOOL,2006,84,47,56%,40,47.6%,85.1%,17,20.2%,36.200000000000003%,23,27.4%,48.9%,7,8.300000000000001%,14.9%,25,29.8%,5,6%
31,Total Cohort,01M515,LOWER EAST SIDE PREPARATORY HIGH SCHO,2006,193,105,54.4%,91,47.2%,86.7%,69,35.799999999999997%,65.7%,22,11.4%,21%,14,7.3%,13.3%,53,27.5%,35,18.100000000000001%


# Converting AP test scores  

We will convert the numerical columns in "ap_2010" from string to numeric data types so that arithmetic operations can be performed on them.  We will then view the data types for all of the columns in "ap_2010" to confirm the correct columns were converted.

In [27]:
# Create list of "ap_2010" columns to convert
ap_columns = ["AP Test Takers ", "Total Exams Taken", "Number of Exams with scores 3 4 or 5"]

# Convert each selected column to a numeric data type
for col in ap_columns:
    data["ap_2010"][col] = pd.to_numeric(data["ap_2010"][col], errors="coerce")
    
# View the data types of each column in "ap_2010"    
data["ap_2010"].dtypes

DBN                                      object
SchoolName                               object
AP Test Takers                          float64
Total Exams Taken                       float64
Number of Exams with scores 3 4 or 5    float64
dtype: object

# Confirming unique DBN values for each dataframe  

We want each dataframe to have a unique DBN value in each row.  To check this, we will print the name of each dataframe along with a list of any multiple occurring DBN values and their counts.

In [28]:
# List any multiple DBN values and their counts for each dataframe
for dataframe in data.keys():
    multiple_dbn = list(data[dataframe]["DBN"].value_counts()[data[dataframe]["DBN"].value_counts() > 1].items())
    print(dataframe, multiple_dbn)

ap_2010 [('04M610', 2)]
class_size []
demographics []
graduation []
hs_directory []
sat_results []
survey []


We can see that "ap_2010" has a DBN value of "04M610" that occurs twice.  We will view that specific DBN value in "ap_2010" to investigate why.

In [29]:
# View rows with "DBN" value of "04M610" in "ap_2010"
data["ap_2010"][data["ap_2010"]["DBN"] == "04M610"]

,DBN,SchoolName,AP Test Takers,Total Exams Taken,Number of Exams with scores 3 4 or 5
51,04M610,THE YOUNG WOMEN'S LEADERSHIP SCHOOL OF EAST HA...,41.0,55.0,29.0
52,04M610,YOUNG WOMEN'S LEADERSHIP SCH,NaN,NaN,NaN


It appears that "ap_2010" has a duplicate row for DBN "04M610" with all null values.  We will drop that row and then confirm there is only 1 instance of DBN "04M610" in "ap_2010".

In [30]:
# Remove the duplicate row in "ap_2010"
data["ap_2010"].drop(index=52, inplace=True)

# View rows with "DBN" value of "04M610" in "ap_2010"
data["ap_2010"][data["ap_2010"]["DBN"] == "04M610"]

,DBN,SchoolName,AP Test Takers,Total Exams Taken,Number of Exams with scores 3 4 or 5
51,04M610,THE YOUNG WOMEN'S LEADERSHIP SCHOOL OF EAST HA...,41.0,55.0,29.0


# Joining all of the dataframes  

All of the dataframes are now ready to be joined into a single dataframe using the "DBN" columns.  To determine which join methods to use, we will first look at the number of rows in each dataframe.

In [34]:
# View the number of rows for each dataframe in the "data" dictionary
for dataframe in data.keys():
    print(dataframe + ": " + str(data[dataframe].shape[0]))

ap_2010: 257
class_size: 583
demographics: 1509
graduation: 405
hs_directory: 435
sat_results: 478
survey: 1702


### Performing the left joins  

The "sat_results" dataframe contains our target variable of SAT scores for each school.  The ultimate purpose for this final cleaned dataframe is to explore any correlations between SAT scores and the other datasets.  Another potential application is predicting SAT scores by training machine learning algorithms on the other data.  Therefore, we want to preserve as much of the "sat_results" data as possible.  

Since "ap_2010" and "graduation" have considerably less rows than "sat_results", we will left join them onto "sat_results" to create a new dataframe named "combined".  The left joins preserve all of the rows from "sat_results" and merges in any rows from "ap_2010" and "graduation" that have a matching DBN value with "sat_results".  

We will view the first 5 rows of the new "combined" dataframe.

In [35]:
# Left merge "ap_2010" and "graduation" onto "sat_results" to create "combined" dataframe
combined = data["sat_results"]
combined = combined.merge(data["ap_2010"], how="left", on="DBN")
combined = combined.merge(data["graduation"], how="left", on="DBN")

# View the first 5 rows of "combined"
combined.head()

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,sat_score,SchoolName,AP Test Takers,Total Exams Taken,Number of Exams with scores 3 4 or 5,Demographic,School Name,Cohort,Total Cohort,Total Grads - n,Total Grads - % of cohort,Total Regents - n,Total Regents - % of cohort,Total Regents - % of grads,Advanced Regents - n,Advanced Regents - % of cohort,Advanced Regents - % of grads,Regents w/o Advanced - n,Regents w/o Advanced - % of cohort,Regents w/o Advanced - % of grads,Local - n,Local - % of cohort,Local - % of grads,Still Enrolled - n,Still Enrolled - % of cohort,Dropped Out - n,Dropped Out - % of cohort
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355.0,404.0,363.0,1122.0,NaN,NaN,NaN,NaN,Total Cohort,HENRY STREET SCHOOL FOR INTERNATIONAL,2006,78.0,43,55.1%,36,46.2%,83.7%,0,0%,0%,36,46.2%,83.7%,7,9%,16.3%,16,20.5%,11,14.1%
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383.0,423.0,366.0,1172.0,UNIVERSITY NEIGHBORHOOD H.S.,39.0,49.0,10.0,Total Cohort,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,2006,124.0,53,42.7%,42,33.9%,79.2%,8,6.5%,15.1%,34,27.4%,64.2%,11,8.9%,20.8%,46,37.1%,20,16.100000000000001%
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377.0,402.0,370.0,1149.0,EAST SIDE COMMUNITY HS,19.0,21.0,NaN,Total Cohort,EAST SIDE COMMUNITY SCHOOL,2006,90.0,70,77.8%,67,74.400000000000006%,95.7%,0,0%,0%,67,74.400000000000006%,95.7%,3,3.3%,4.3%,15,16.7%,5,5.6%
3,01M458,FORSYTH SATELLITE ACADEMY,7,414.0,401.0,359.0,1174.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01M509,MARTA VALLE HIGH SCHOOL,44,390.0,433.0,384.0,1207.0,NaN,NaN,NaN,NaN,Total Cohort,MARTA VALLE HIGH SCHOOL,2006,84.0,47,56%,40,47.6%,85.1%,17,20.2%,36.200000000000003%,23,27.4%,48.9%,7,8.300000000000001%,14.9%,25,29.8%,5,6%


### Performing the inner joins  

... Even though "hs_directory" has less rows than "sat_results" we still only want to do an inner join because we will be mapping the data

In [ ]:
data.keys()

In [ ]:
join_keys = ['class_size', 'demographics', 'survey', 'hs_directory']

for key in join_keys:
    combined = combined.merge(data[key], how='inner', on='DBN')
    
combined.shape

In [ ]:
combined.head()

# 15. Filling in Missing Values

In [ ]:
type(combined.mean())

In [ ]:
means = combined.mean()
means

In [ ]:
combined.fillna(means, inplace=True)
combined

In [ ]:
combined.fillna(0, inplace=True)
print(combined.shape)
combined.head()

# 16. Adding a School District Column for Mapping

In [ ]:
'Sinbad'[:2]

In [ ]:
def first_two(string):
    return string[:2]

In [ ]:
combined['school_dist'] = combined['DBN'].apply(first_two)
combined[['DBN', 'school_dist']].head()

# Mission 3 Data Cleaning Walkthrough: Analyzing and Visualizing Data

# 3. Finding Correlations with the r Value

In [ ]:
correlations = combined.corr()
correlations

In [ ]:
correlations = correlations['sat_score']
correlations

In [ ]:
correlations['total_enrollment']

# 5. Plotting Enrollment with the Plot() Accessor

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
combined.plot(x='total_enrollment', y='sat_score', kind='scatter')
correlations['total_enrollment']

# 6. Exploring Schools With Low SAT Scores and Enrollment

In [ ]:
low_enrollment = combined[combined['total_enrollment'] < 1000]
low_enrollment = combined[combined['sat_score'] < 1000]
print(low_enrollment.shape)
low_enrollment[['School Name', 'total_enrollment', 'sat_score']]

# 7. Plotting Language Learning Percentage

In [ ]:
combined.plot(x='ell_percent', y='sat_score', kind='scatter')
correlations['ell_percent']

# 8. Mapping the Schools With Basemap

In [ ]:
from mpl_toolkits.basemap import Basemap

In [ ]:
m = Basemap(
    projection='merc', 
    llcrnrlat=40.496044, 
    urcrnrlat=40.915256, 
    llcrnrlon=-74.255735, 
    urcrnrlon=-73.700272,
    resolution='l'
)

m.drawmapboundary(fill_color='#85A6D9')
m.drawcoastlines(color='#6D5F47', linewidth=.4)
m.drawrivers(color='#6D5F47', linewidth=.4)

# 9. Mapping the Schools With Basemap

In [ ]:
m = Basemap(
    projection='merc', 
    llcrnrlat=40.496044, 
    urcrnrlat=40.915256, 
    llcrnrlon=-74.255735, 
    urcrnrlon=-73.700272,
    resolution='l'
)

m.drawmapboundary(fill_color='#85A6D9')
m.drawcoastlines(color='#6D5F47', linewidth=.4)
m.drawrivers(color='#6D5F47', linewidth=.4)

longitudes = combined['lon'].tolist()
latitudes = combined['lat'].tolist()

m.scatter(longitudes, latitudes, s=20, zorder=2, latlon=True)

# 10. Plotting Out Statistics

In [ ]:
m = Basemap(
    projection='merc', 
    llcrnrlat=40.496044, 
    urcrnrlat=40.915256, 
    llcrnrlon=-74.255735, 
    urcrnrlon=-73.700272,
    resolution='l'
)

m.drawmapboundary(fill_color='#85A6D9')
m.drawcoastlines(color='#6D5F47', linewidth=.4)
m.drawrivers(color='#6D5F47', linewidth=.4)

longitudes = combined['lon'].tolist()
latitudes = combined['lat'].tolist()

m.scatter(longitudes, latitudes, s=20, zorder=2, latlon=True, c=combined['ell_percent'], cmap='summer')

# 11. Calculating District Level Statistics

In [ ]:
districts = combined.groupby('school_dist').agg(np.mean)
districts

In [ ]:
districts.reset_index(inplace=True)
districts.head()

# 12. Plotting Percent Of English Learners by District

In [ ]:
m = Basemap(
    projection='merc', 
    llcrnrlat=40.496044, 
    urcrnrlat=40.915256, 
    llcrnrlon=-74.255735, 
    urcrnrlon=-73.700272,
    resolution='l'
)

m.drawmapboundary(fill_color='#85A6D9')
m.drawcoastlines(color='#6D5F47', linewidth=.4)
m.drawrivers(color='#6D5F47', linewidth=.4)

longitudes = districts['lon'].tolist()
latitudes = districts['lat'].tolist()

m.scatter(longitudes, latitudes, s=50, zorder=2, latlon=True, c=districts['ell_percent'], cmap='summer')

# Guided Project: Analyzing NYC High School Data

# 1. Introduction

In [ ]:
survey_fields_with_SAT = [
    "DBN",
    'sat_score',
    "rr_s", 
    "rr_t", 
    "rr_p", 
    "N_s", 
    "N_t", 
    "N_p", 
    "saf_p_11", 
    "com_p_11", 
    "eng_p_11", 
    "aca_p_11", 
    "saf_t_11", 
    "com_t_11", 
    "eng_t_11", 
    "aca_t_11", 
    "saf_s_11", 
    "com_s_11", 
    "eng_s_11", 
    "aca_s_11", 
    "saf_tot_11", 
    "com_tot_11", 
    "eng_tot_11", 
    "aca_tot_11",
    ]

In [ ]:
combined.loc[:,survey_fields_with_SAT]

In [ ]:
corr_survey_sat = combined.loc[:,survey_fields_with_SAT].corr()
corr_survey_sat = corr_survey_sat['sat_score'][1:]
corr_survey_sat

In [ ]:
corr_survey_sat.plot.barh()

### Findings

Mine: Academic expectations and safety and respect scores have strong r-values with SAT scores. Makes sense, hard to learn in an unsafe environment, and I'd expect high achieveing students with high expectations of a school to perform well academically at that school.

Solution: There are high correlations between N_s, N_t, N_p and sat_score. Since these columns are correlated with total_enrollment, it makes sense that they would be high.

It is more interesting that rr_s, the student response rate, or the percentage of students that completed the survey, correlates with sat_score. This might make sense because students who are more likely to fill out surveys may be more likely to also be doing well academically.

How students and teachers percieved safety (saf_t_11 and saf_s_11) correlate with sat_score. This make sense, as it's hard to teach or learn in an unsafe environment.

The last interesting correlation is the aca_s_11, which indicates how the student perceives academic standards, correlates with sat_score, but this is not true for aca_t_11, how teachers perceive academic standards, or aca_p_11, how parents perceive academic standards.

# 2. Exploring Safety and SAT Scores

In [ ]:
combined.plot.scatter(x='saf_s_11', y='sat_score')

### Findings

Mine: Doesn't appear to be a strong linear correlations, weak at best.

Solution: There appears to be a correlation between SAT scores and safety, although it isn't that strong. It looks like there are a few schools with extremely high SAT scores and high safety scores. There are a few schools with low safety scores and low SAT scores. No school with a safety score lower than 6.5 has an average SAT score higher than 1500 or so.

In [ ]:
combined.head()

In [ ]:
school_dist = combined.groupby('school_dist').agg(np.mean)
school_dist.reset_index(inplace=True)
school_dist

In [ ]:
m = Basemap(
    projection='merc', 
    llcrnrlat=40.496044, 
    urcrnrlat=40.915256, 
    llcrnrlon=-74.255735, 
    urcrnrlon=-73.700272,
    resolution='l'
)

m.drawmapboundary(fill_color='#85A6D9')
m.drawcoastlines(color='#6D5F47', linewidth=.4)
m.drawrivers(color='#6D5F47', linewidth=.4)

longitudes = school_dist['lon'].tolist()
latitudes = school_dist['lat'].tolist()

m.scatter(longitudes, latitudes, s=50, zorder=2, latlon=True, c=school_dist['saf_s_11'], cmap='summer')

### Findings

Mine: Safest schools are in Manhattan as expected. With exception of a few, unsafest schools are in Brokklyn, Queens and the Bronx

Solution: It looks like Upper Manhattan and parts of Queens and the Bronx tend to have lower safety scores, whereas Brooklyn has high safety scores.

# 3. Exploring Race and SAT Scores

In [ ]:
race_cols = ['sat_score', 'white_per', 'asian_per', 'black_per', 'hispanic_per']

In [ ]:
race_sat = combined[race_cols].corr()['sat_score'][1:]
race_sat

In [ ]:
race_sat.plot.barh()

### Findings

Mine: Asian and White are strongly positively correlated. Hispanic and Black are strongly negatively correlated.  Possibly suggests racial bias in test, or indicative of lack of funding for schools in areas with high % of hispanic and black students.

Solution: It looks like a higher percentage of white or asian students at a school correlates positively with sat score, whereas a higher percentage of black or hispanic students correlates negatively with sat score. This may be due to a lack of funding for schools in certain areas, which are more likely to have a higher percentage of black or hispanic students.

In [ ]:
combined.plot.scatter('hispanic_per', 'sat_score')

### Findings

Mine: appears to be negative correlation, although not that strong. Schools with greater than approx 25% hispanic don't have an SAT score above around 1500

In [ ]:
combined.head()

In [ ]:
combined[combined['hispanic_per'] > 95]['SCHOOL NAME']

### Findings

Solution: The schools listed above appear to primarily be geared towards recent immigrants to the US. These schools have a lot of students who are learning English, which would explain the lower SAT scores.

In [ ]:
hisp_10_1800 = combined[combined['hispanic_per'] < 10]
hisp_10_1800[hisp_10_1800['sat_score'] > 1800]['SCHOOL NAME']

### Findings

Mine: all appear to be science and technical schools, suggesting higher acceptance requirements, and hence higher achieving students

Solution: Many of the schools above appear to be specialized science and technology schools that receive extra funding, and only admit students who pass an entrance exam. This doesn't explain the low hispanic_per, but it does explain why their students tend to do better on the SAT -- they are students from all over New York City who did well on a standardized test.

# 4. Exploring Gender and SAT Scores

In [ ]:
sex_cols = ['sat_score', 'male_per', 'female_per']

In [ ]:
sex_sat = combined[sex_cols].corr()['sat_score'][1:]
sex_sat

In [ ]:
sex_sat.plot.bar()

### Findings

Mine: females positive corr, males negative corr, but neither are strong.

Solution: In the plot above, we can see that a high percentage of females at a school positively correlates with SAT score, whereas a high percentage of males at a school negatively correlates with SAT score. Neither correlation is extremely strong.

In [ ]:
combined.plot.scatter('female_per', 'sat_score')

### Findings

Mine: don't see any strong linear corr.  There's high and low sat scores with high and low female_per

Solution: Based on the scatterplot, there doesn't seem to be any real correlation between sat_score and female_per. However, there is a cluster of schools with a high percentage of females (60 to 80), and high SAT scores.

In [ ]:
fem_60_1700 = combined[combined['female_per'] > 60]
fem_60_1700[fem_60['sat_score'] > 1700]['SCHOOL NAME']

### Findings

Solution: These schools appears to be very selective liberal arts schools that have high academic standards

# 5. Exploring AP Scores vs. SAT Scores

In [ ]:
combined['ap_per'] = combined['AP Test Takers '] / combined['total_enrollment']
combined['ap_per']

In [ ]:
combined.plot.scatter('ap_per', 'sat_score')

### Findings

Mine: some corr but not strong. There is a cluster of schools with ap_per > .4 with high SAT, but also a bunch with very high ap_per with SAT no greater than 1200

Solution: It looks like there is a relationship between the percentage of students in a school who take the AP exam, and their average SAT scores. It's not an extremely strong correlation, though.